In [1]:
!wget https://www.gutenberg.org/files/11/11-0.txt

--2019-07-26 16:25:42--  https://www.gutenberg.org/files/11/11-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173595 (170K) [text/plain]
Saving to: ‘11-0.txt’

11-0.txt            100%[===================>] 169.53K  --.-KB/s    in 0.1s    

2019-07-26 16:25:43 (1.14 MB/s) - ‘11-0.txt’ saved [173595/173595]



In [0]:
data = open('11-0.txt').read()

In [0]:
data = data.split("*** START OF THIS PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***")[1]

In [0]:
data = data.split("THE END")[0]

In [7]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
import string


# load ascii text and covert to lowercase and remove punctuations
raw_text = data.lower().translate(str.maketrans('', '', string.punctuation))
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 500
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
X = pad_sequences(X)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.1))
model.add(LSTM(256))
# model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=100, batch_size=512, callbacks=callbacks_list)

Total Characters:  139103
Total Vocab:  34
Total Patterns:  138603
Epoch 1/100
138603/138603 [==============================] - 361s 3ms/step - loss: 2.9521

Epoch 00001: loss improved from inf to 2.95215, saving model to weights-improvement-01-2.9521-bigger.hdf5
Epoch 2/100
138603/138603 [==============================] - 354s 3ms/step - loss: 2.9344

Epoch 00002: loss improved from 2.95215 to 2.93437, saving model to weights-improvement-02-2.9344-bigger.hdf5
Epoch 3/100
138603/138603 [==============================] - 352s 3ms/step - loss: 2.9337

Epoch 00003: loss improved from 2.93437 to 2.93368, saving model to weights-improvement-03-2.9337-bigger.hdf5
Epoch 4/100
138603/138603 [==============================] - 352s 3ms/step - loss: 2.9330

Epoch 00004: loss improved from 2.93368 to 2.93305, saving model to weights-improvement-04-2.9330-bigger.hdf5
Epoch 5/100
138603/138603 [==============================] - 352s 3ms/step - loss: 2.9325

Epoch 00005: loss improved from 2.93305 to

KeyboardInterrupt: ignored

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [12]:
!cp weights-improvement-44-2.9308-bigger.hdf5 /content/gdrive/My Drive/Colab Notebooks/text_gen_weights-improvement-44-2.9308-bigger.hdf5

cp: target 'Drive/Colab Notebooks/text_gen_weights-improvement-44-2.9308-bigger.hdf5' is not a directory


In [13]:
from shutil import copyfile
copyfile("weights-improvement-44-2.9308-bigger.hdf5", "/content/gdrive/My Drive/Colab Notebooks/text_gen_weights-improvement-44-2.9308-bigger.hdf5")

'/content/gdrive/My Drive/Colab Notebooks/text_gen_weights-improvement-44-2.9308-bigger.hdf5'

In [0]:
model.save_weights("/content/gdrive/My Drive/Colab Notebooks/text_gen_weights-improvement-44-2.9308-bigger_weights.hdf5")

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [9]:
# Larger LSTM Network to Generate Text for Alice in Wonderland
import numpy
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
import string


# load ascii text and covert to lowercase and remove punctuations
raw_text = data.lower().translate(str.maketrans('', '', string.punctuation))
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 500
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
X = pad_sequences(X)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.1))
model.add(LSTM(256))
# model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
model = load_model('/content/gdrive/My Drive/Colab Notebooks/text_gen_weights-improvement-44-2.9308-bigger.hdf5')
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=512, callbacks=callbacks_list)

Total Characters:  139103
Total Vocab:  34
Total Patterns:  138603


W0726 16:31:18.498258 140412088608640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 16:31:18.500167 140412088608640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0726 16:31:18.512406 140412088608640 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0726 16:31:18.529063 140412088608640 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backe

Epoch 1/50
138603/138603 [==============================] - 353s 3ms/step - loss: 2.9308

Epoch 00001: loss improved from inf to 2.93082, saving model to weights-improvement-01-2.9308-bigger.hdf5
Epoch 2/50
138603/138603 [==============================] - 353s 3ms/step - loss: 2.9310

Epoch 00002: loss did not improve from 2.93082
Epoch 3/50
138603/138603 [==============================] - 355s 3ms/step - loss: 2.9309

Epoch 00003: loss did not improve from 2.93082
Epoch 4/50
138603/138603 [==============================] - 356s 3ms/step - loss: 2.9309

Epoch 00004: loss did not improve from 2.93082
Epoch 5/50
138603/138603 [==============================] - 356s 3ms/step - loss: 2.9309

Epoch 00005: loss did not improve from 2.93082
Epoch 6/50
138603/138603 [==============================] - 358s 3ms/step - loss: 2.9309

Epoch 00006: loss did not improve from 2.93082
Epoch 7/50
138603/138603 [==============================] - 359s 3ms/step - loss: 2.9309

Epoch 00007: loss did not imp

In [11]:
# Load LSTM network and generate text
import sys
import numpy
import glob
import os
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import string
# load ascii text and covert to lowercase
# filename = "wonderland.txt"
# raw_text = open(filename).read()
raw_text = data
raw_text = data.lower().translate(str.maketrans('', '', string.punctuation))

# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 500
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.1))
model.add(LSTM(256))
# model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))
# load the network weights
filename = "weights-improvement-19-1.9435.hdf5"
# model.load_weights(filename)
model = load_model(max(glob.glob('*'), key=os.path.getctime))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Total Characters:  139103
Total Vocab:  34
Total Patterns:  138603
Seed:
" f and that in
about half no time take your choice’

the duchess took her choice and was gone in a moment

‘let’s go on with the game’ the queen said to alice and alice was
too much frightened to say a word but slowly followed her back to the
croquetground

the other guests had taken advantage of the queen’s absence and were
resting in the shade however the moment they saw her they hurried
back to the game the queen merely remarking that a moment’s delay would
cost them their lives

all the time  "
                                                                                                                                                                                                                                                                                

KeyboardInterrupt: ignored